In [1]:
import numpy as np
from Tomita_Grammars import tomita_3
from Training_Functions import make_train_set_for_target
import tensorflow as tf

print(f'Tensorflow: {tf.__version__}')


Tensorflow: 2.10.0


In [2]:
target = tomita_3
alphabet = "01"

In [3]:
train_set = make_train_set_for_target(target,alphabet)

print(len(train_set))
print(list(train_set.items())[:10])

made train set of size: 2748 , of which positive examples: 1325
2748
[('', True), ('0', True), ('1', True), ('01', True), ('11', True), ('00', True), ('10', False), ('110', True), ('111', True), ('100', True)]


In [4]:
words = list(train_set.keys())
target = list(train_set.values())
print(words[:10])
print(target[:10])
target = [int(i) for i in target]
print(target[:10])


['', '0', '1', '01', '11', '00', '10', '110', '111', '100']
[True, True, True, True, True, True, False, True, True, True]
[1, 1, 1, 1, 1, 1, 0, 1, 1, 1]


In [5]:
char_set = ['<pad>'] + list(alphabet)
idx2char = {idx:char for idx, char in enumerate(char_set)}
char2idx = {char:idx for idx, char in enumerate(char_set)}
char2idx

{'<pad>': 0, '0': 1, '1': 2}

In [6]:
X = list(map(lambda word: [char2idx.get(char) for char in word], words))
X_len = list(map(lambda word: len(word), X))

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence = 30

X = pad_sequences(X, maxlen=max_sequence, padding='post', truncating='post')

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((X, target)).shuffle(buffer_size=4).batch(batch_size=50)
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 30), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [9]:
input_dim = len(char2idx)
output_dim = len(char2idx)

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from SecondDegree import SecondDegreeCell

model = Sequential()
model.add(layers.Embedding(input_dim=input_dim, output_dim=output_dim, mask_zero=True,
             input_length=max_sequence, trainable=False, 
             embeddings_initializer=tf.keras.initializers.random_normal()))
model.add(layers.RNN(SecondDegreeCell(10)))
model.add(layers.Dense(2))

    



model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 3)             9         
                                                                 
 rnn (RNN)                   (None, 10)                310       
                                                                 
 dense (Dense)               (None, 2)                 22        
                                                                 
Total params: 341
Trainable params: 332
Non-trainable params: 9
_________________________________________________________________


In [11]:
def loss_fn(model, X, y):
    return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_true=y, 
                                                                          y_pred=model(X), 
                                                                          from_logits=True))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [12]:
tr_loss_hist = []

for e in range(250):
    avg_tr_loss = 0
    tr_step = 0
    
    for x_mb, y_mb in train_ds:
        with tf.GradientTape() as tape:
            tr_loss = loss_fn(model, x_mb, y_mb)
            
        grads = tape.gradient(tr_loss, sources=model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
        avg_tr_loss += tr_loss
        tr_step += 1
    
    avg_tr_loss /= tr_step
    tr_loss_hist.append(avg_tr_loss)
    
    if (e + 1) % 5 == 0:
        print('epoch: {:3}, tr_loss: {:3f}'.format(e + 1, avg_tr_loss))

epoch:   5, tr_loss: 0.697136
epoch:  10, tr_loss: 0.693413
epoch:  15, tr_loss: 0.641204
epoch:  20, tr_loss: 0.619596
epoch:  25, tr_loss: 0.612444
epoch:  30, tr_loss: 0.607704
epoch:  35, tr_loss: 0.604628
epoch:  40, tr_loss: 0.602374
epoch:  45, tr_loss: 0.600474
epoch:  50, tr_loss: 0.597893
epoch:  55, tr_loss: 0.594824
epoch:  60, tr_loss: 0.591238
epoch:  65, tr_loss: 0.587961
epoch:  70, tr_loss: 0.584938
epoch:  75, tr_loss: 0.582311
epoch:  80, tr_loss: 0.580053
epoch:  85, tr_loss: 0.578240
epoch:  90, tr_loss: 0.576728
epoch:  95, tr_loss: 0.575432
epoch: 100, tr_loss: 0.574328
epoch: 105, tr_loss: 0.573372
epoch: 110, tr_loss: 0.572492
epoch: 115, tr_loss: 0.571686
epoch: 120, tr_loss: 0.570945
epoch: 125, tr_loss: 0.570305
epoch: 130, tr_loss: 0.569473
epoch: 135, tr_loss: 0.568709
epoch: 140, tr_loss: 0.568012
epoch: 145, tr_loss: 0.567268
epoch: 150, tr_loss: 0.566454
epoch: 155, tr_loss: 0.565762
epoch: 160, tr_loss: 0.564969
epoch: 165, tr_loss: 0.564057
epoch: 170

In [13]:
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=-1)

86/86 [==============================] - 0s 2ms/step


In [14]:
print(f'acc: {np.mean(y_pred == target):.2%}')

acc: 74.02%
